## 절차

|절차|단계|목표/설명|
|-|-|-|
|1|연구목표|사용자가 입력한 텍스트를 예측하여 어떤 언어인지 판독한다(영어권)<br>머신러닝의 사용 여부 판단(굳이 사용 안해도 처리 가능하면 제외<br>머신러닝을 적용, 지도학습법을 사용<br>알고리즘은 여러 개를 테스트하여 최적화가 된 알고리즘을 사용(파라미터 적용)|
|2|데이터 획득/수집|실전 : 위키피디아/소설/법률 등 텍스트 크롤링<br>임시 : 제공데이터(법령/대본 등)|
|3|데이터 준비/통찰/전처리|텍스트에서 알파벳의 출현 빈도를 계산하여 벡터화<br>훈련데이터/테스트데이터 구성|
|4|데이터 탐색/통찰/시각화|언어별로 알파벳이 출현하는(or 알파벳을 사용하는) 빈도가 다르다는 명제를 확인<br>시각화를 진행<br>바차트를 국가별로 비교, 선형차트를 통해서 빈도의 차이점 시각화|
|5|데이터 모델링 or 모델구축|- 학습<br>- 예측<br>- 정답률<br>- 성능평가<br>- 알고리즘체인/파이프라인 구축<br>- 하이퍼파라미터 튜닝<br>- 연구목표에서 설정한 목표치까지 도달할 때까지 반복|
|6|시스템 통합 or 솔루션/서비스/레포트|- 모델을 덤프 → flask 웹 서비스 기반에 연동 → 서비스 구성<br>- 모델을 업그레이드/다운그레이드가 자동화될 수 있게 구성<br>- 선순환구조(사용자가 사용하면 할수록 그 데이터가 다시 모여서 학습 → 성능을 향상 → 업데이트|

### 1. 연구목표
- 사용자가 입력한 텍스트를 예측하여 어떤 언어인지 판독한다(영어권)

### 2. 데이터 획득/수집
- 영어 샘플 사이트 : https://en.wikipedia.org/wiki/BTS_(band)
- #mw-content-text p

In [2]:
# 2-1. 실전용 : 미국, 프랑스, 인도네시아, 타갈리아어 등의 위키피디아의 텍스트 수집
#               수집된 정량은 비슷한 분량으로 준비
import urllib.request as req
from bs4 import BeautifulSoup

In [3]:
# 2-2. 요청 및 soup로 구성
target_url = 'https://en.wikipedia.org/wiki/BTS_(band)'
soup = BeautifulSoup( req.urlopen( target_url ), 'html5lib' )

In [5]:
# soup
type(soup)

bs4.BeautifulSoup

In [7]:
# 2-3. 테이터 추출
output = soup.select('#mw-content-text p')
len(output)

64

In [9]:
# 2-4. 리스트에 모든 텍스트를 담아라(멤버로)
# for p in output → p tag가 하나씩 나온다
texts = [ p.text for p in output ]
texts[:3]

['\n\n',
 '\n',
 'BTS (Korean:\xa0방탄소년단; RR:\xa0Bangtan Sonyeondan), also known as the Bangtan Boys, is a seven-member South Korean boy band formed in Seoul in 2013. The septet co-writes and produces much of their output. Originally a hip hop group, their musical style has evolved to include a wide range of genres. Their lyrics, often focused on personal and social commentary, touch on the themes of mental health, troubles of school-age youth, loss, the journey towards loving oneself, and individualism. Their work features references to literature and psychological concepts and includes an alternative universe storyline. Popular for their live performances, the group have staged several world tours.\n']

In [12]:
import re

In [15]:
# 2-5. 정규식 처리
# 통합, 조인 ↔ 분해(의 결과물은 리스트)
str_text = ' '.join(texts)
str_text[:20]
# 영어, 공백(단어 구분) 빼고 다 제거
p = re.compile('[^a-zA-Z ]')
m = p.sub( '', str_text )
print(m[:50])

  BTS Korean RRBangtan Sonyeondan also known as th


In [ ]:
# 위와 같은 방식으로 데이터를 언어별로 수집
# → 대량의 데이터를 확보한다 (충분한 학습을 수행하기 위한 재료)
# 이렇게 모아진 데이터를 사용하여 학습 진행

### 3. 데이터 준비/통찰/전처리 + 시각화
- 제공되는 모든 데이터를 읽어서 처리
- 알파벳의 빈도를 계산하여 수치화

In [16]:
# 파일 목록 리스트
import glob, os.path, re, json

file_list = glob.glob('../data/train/*.txt')
file_list

['../data/train\\en-1.txt',
 '../data/train\\en-2.txt',
 '../data/train\\en-3.txt',
 '../data/train\\en-4.txt',
 '../data/train\\en-5.txt',
 '../data/train\\fr-10.txt',
 '../data/train\\fr-6.txt',
 '../data/train\\fr-7.txt',
 '../data/train\\fr-8.txt',
 '../data/train\\fr-9.txt',
 '../data/train\\id-11.txt',
 '../data/train\\id-12.txt',
 '../data/train\\id-13.txt',
 '../data/train\\id-14.txt',
 '../data/train\\id-15.txt',
 '../data/train\\tl-16.txt',
 '../data/train\\tl-17.txt',
 '../data/train\\tl-18.txt',
 '../data/train\\tl-19.txt',
 '../data/train\\tl-20.txt']

In [24]:
# 파일을 읽어서
# 빈도 계산
# 지도 방식 데이터로 구성 : 변수(a-z) + 답안(1개)

def check_lang_freq ( file_path ) :
    
    # Section 1
    # --------------------------------------------------------------------------
    # 1. 정답 변수(컬럼)를 구성
    # '../data/train\\en-1.txt' → 'en'(정답) 추출
    # file_path를 통찰해보니, 위치가 변경되면 정규식이 적용이 안 될 수 있다
    # 파일명을 추출하여 거기서 정규식을 적용하는 것이 합리적 → 경로가 바뀌어도 ok
    name = os.path.basename( file_path )
    # print(name)    # en-1.txt
    p = re.compile('^[a-z]{2,}')
    lang = p.match(name).group()
    # print(name, lang)
    # -------------------------------------------------------------------------------
    
    # Section 2
    # -------------------------------------------------------------------------------
    # 알파벳 빈도 계산
    # 2. 파일을 오픈 → 읽고 → 닫기 → [0, 0, 0, 0, 0, 0, 0, 0, 0, ,,, 0]
    with open ( file_path, 'r', encoding='utf-8') as f :
        # 데이터를 들여다보니 대소문자가 섞여 있다
        # 대소문자는 아스키 값이 다르니 한쪽 방향으로 통일해야 한다
        # 여기서는 소문자로 통일
        text = f.read().lower()
        p = re.compile('[^a-zA-Z]*')
        text = p.sub('', text)
    # 알파벳 순서대로 빈도수 기록
    # a-a = 0, b-a = 1, c-a = 2, ... z-a = 25 → 빈도를 기록할 인덱스와 동일
    cnts = [ 0 for n in range(26) ]
    # 아스키 환산함수 : ord()
    # print(ord('a'), ord('z'), ord(' ')) : 97 122 32
    asc_a, asc_z, asc_ws = ord('a'), ord('z'), ord(' ')
    for ch in text :
        # ch를 아스키로 변환
        # if asc_a <= ord(ch) <= asc_z :  # 알파벳이 맞으면
        if ord(ch) != asc_ws :
            cnts[ord(ch)-asc_a] += 1
    # -------------------------------------------------------------------------------

    # Section 3
    # -------------------------------------------------------------------------------
    # 3. 빈도는 파일 한 개 내 전체 charater 수 대비 해당 a~z까지의 빈도수로 표현
    # → 데이터를 같은 구간에 배치 → 정규화 0 ~ 1 사이에 배치
    # 전체 빈도
    total_cnt = sum(cnts)
    freq = list( map( lambda x : x/total_cnt , cnts) )
    # -------------------------------------------------------------------------------

    # Section 4
    # -------------------------------------------------------------------------------
    # 4. 리턴 : 정답, 빈도를 담는 리스트
    # -------------------------------------------------------------------------------
    
    return lang, freq

In [26]:
# for file in file_list :
#     print(check_lang_freq(file))

In [27]:
# 특정 폴더 내의 데이터를 읽어서 빈도로 리턴
def load_files( file_glob ) :
    file_list = glob.glob( file_glob )
    langs = []
    freqs = []
    for file in file_list :
        lang, freq = check_lang_freq(file)
        langs.append(lang)
        freqs.append(freq)
    # 딕셔너리 형태로 리턴
    return { 'labels' : langs, 'freqs' : freqs }

In [28]:
# 데이터 로드
train_data = load_files('../data/train/*.txt')
test_data = load_files('../data/test/*.txt')

In [29]:
# 데이터 확인
len(train_data['labels']), len(test_data['labels'])

(20, 8)

In [30]:
# 데이터 적재
# 자료구조를 그대로 유지해서 저장 → json 모듈로 처리
with open( '../data/lang_label_freq.json', 'w', encoding='utf-8' ) as f :
    json.dump( [train_data, test_data], f )

### 4. 데이터 탐색/통찰/시각화

### 5. 데이터 모델링 or 모델구축 + 시각화
- 알고리즘 선택
- 훈련용 데이터, 테스트용 데이터 구성
- 훈련
- 예측모델 생성
- 새로운 데이터를 적용해서 예측
- 정확도 판단, 검증
--------------------------------------------------------------
- 파이프라인 구축
- 하이퍼 파라미터 튜닝
- 성능 평가

### 6. 시스템 통합 or 솔루션/서비스/레포트
- flask
- mariadb : 데이터 축적, 예측 모델 데이터 업그레이드/다운그레이드 정보